In [0]:
%md
1. check count: distinct count group by process date (find the latest data), compared with topic data and  load to DB data:
topic data, please use delta lake to read
`s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/ `
load to db data:
`s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/`
            a.  PRE aa.adv_join_201908_20200116_1/
            b.  PRE aa.adv_join_20200116_2020024_2/
            c.  PRE aa.adv_join_20200220_202003_3/
            d. `s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200306_20200310/`
2. Pick data in a, b, c , compared 10 review details of each folder:
		    a.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/`
		    b.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/` + where (process date between 2020-01-16 and 2020-02-24 )
            c.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/` + where ( process date between 2020-02-20 and 2020-03-07 ) , use delta lake
            d.  review data:   `s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/` + where ( process date between 2020-03-07 and 2020-03-10 ) , use delta lake
3. Find not joined data topic:
write not join data with review_id, topic, process date, event date, to
`s3://b2c-prod-data-pipeline-qa/aa.review/adv_not_join/`

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/" 

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/"

In [0]:
%%sh


In [0]:

topic_test_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"
df_topic_test = spark.read.format('delta').load(topic_test_path)
df_topic_test.show(4, True, True)

In [0]:

load_test_paths = ["s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_201908_20200116_1/event_month=*", \
                   "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200116_2020024_2/event_month=*", \
                   "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200220_202003_3/event_month=*", \
                   "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200306_20200310/event_month=*"]
df_load_test = spark.read.parquet(*load_test_paths)
df_load_test.show(4, True, True)

In [0]:

print("Before grouped topic data:", df_topic_test.count())
df_topic_grouped = df_topic_test.groupby("review_id").agg({"process_date": "max"})
df_topic_grouped_count = df_topic_grouped.count()
print("After grouped topic data:", df_topic_grouped_count)
df_topic_selected = df_topic_grouped.select("review_id")

df_load_selected = df_load_test.select("review_id")

df_except = df_topic_selected.subtract(df_load_selected)

In [0]:

df_topic_test2 = df_topic_test.select("review_id", "process_date", "date", "topic_ids").withColumnRenamed('review_id', 'key')
df_not_joined = df_topic_test2.join(df_except, df_topic_test2.key==df_except.review_id, 'right_outer').select('review_id', 'process_date', 'date', 'topic_ids')

df_not_joined.show(3)

a = df_load_test.where("review_id='5400356011'")
a.show()

In [0]:

df_not_joined.coalesce(1).write.mode("overwrite").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/adv_not_join")

In [0]:

df_not_joined_test = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review/adv_not_join/")
df_not_joined_test.show(3)
df_not_joined.groupby("process_date").count().show(100)

In [0]:

df_dev_not_join = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_not_join_201907_20200310_not_fix/")
print df_dev_not_join.count()
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_not_join_201907_20200310_fix/").count()
print df_dev_not_join.printSchema()

In [0]:

print("After subtracting", df_except.count())

df_anti_except = df_load_selected.subtract(df_topic_selected)
print("anti subtracting", df_anti_except.count())

In [0]:

df = spark.createDataFrame([('111', '2020-02-01'),('222', '2020-02-02'), ('333', '2020-02-03'), ('222', '2020-02-03')],["review_id", "date"])
df.show()
# df.groupby("review_id").agg({"date": "max"}).show()
df2 = spark.createDataFrame([('111', '2020-02-01'), ('444', '2020-02-20')],["review_id", "date"])
df2 = df2.select("review_id")
df2 = df2.withColumnRenamed("review_id", "key")
df2.show()

df3 = df.join(df2, df.review_id==df2.key, 'right').select("key", "date")
df3.show()

# print df[df.review_id.isin(['111'])].collect()
# df3 = df.subtract(df2)
# df3.show()

df.head(1)

In [0]:
%md
1. check count: distinct count group by process date (find the latest data), compared with topic data and  load to DB data:
topic data, please use delta lake to read
`s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/ `
load to db data:
`s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/`
            a.  PRE aa.adv_join_201908_20200116_1/
            b.  PRE aa.adv_join_20200116_2020024_2/
            c.  PRE aa.adv_join_20200220_202003_3/
2. Pick data in a, b, c , compared 10 review details of each folder:
		    a.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/`
		    b.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/` + where (process date between 2020-01-16 and 2020-02-24 )
            c.  review data:    `s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/` + where ( process date between 2020-02-20 and 2020-03-07 ) , use delta lake
3. Find not joined data topic:
write not join data with review_id, topic, process date, event date, to
`s3://b2c-prod-data-pipeline-qa/aa.review/adv_not_join/`

In [0]:
%md
## Review Data ------------------------------------------------------------------------------------------

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2019-06/ --recursive --human --summarize | tail -5

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/

In [0]:
%md
## Load to DB Data------------------------------------------------------------------------------------------

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_201908_20200116_1/event_month=2019-06/ --recursive --human --summarize | tail -2


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200116_2020024_2/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200220_202003_3/

In [0]:

# df_b = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-21/')
spark.read.format("delta").parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("review_id='5549512716'").show()
# df_b.where("process_granularity='daily' and process_date='2017-06-10' and process_hour=23 and device_code='ios-all' and market_code='apple-store'")
# df_b.show(4, True, True)
# row_10_load_data = df_load_test.take(10)
# df_load_test.show(10, True, True)
# review_id_list = [row['review_id'] for row in row_10_load_data]
# for review_id in review_id_list:
# a = df_b.where("review_id='4866938646'")
# print a
# df_b.show(2, True, True)

In [0]:

a.show()

In [0]:

def detail_check(num, picked_data_path, delta1, source_data_path, delta2, condition=None):
    def read_data(path, delta, condition):
        '''
        check data format (delta lake or not)
        '''
        if delta:
            return spark.read.format('delta').load(path).where(condition)
        return spark.read.parquet(path).where(condition)
        
    # Read data from s3 path
    df_picked_data = read_data(picked_data_path, delta1, condition)
    df_source_data = read_data(source_data_path, delta2, condition) 
    
    # Review id list from picked data
    row_picked_data = df_picked_data.take(10)
    
    print "row_picked_data:", row_picked_data
    
    review_id_list = [row['review_id'] for row in row_picked_data]
    
    print "review_id_list:", review_id_list
    
    # Search picked data in source data
    row_source_data = []
    for review_id in review_id_list:
        print "review_id = ", review_id
        row_source_data.append(df_source_data.where("review_id='{}'".format(review_id)).collect()[0])
        print row_source_data
    
    return df_picked_data, df_source_data, row_picked_data, row_source_data




load_data_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_201908_20200116_1/"
review_data_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/"
condition = "event_month='2019-06'"
df_picked_data, df_source_data, row_picked_data, row_source_data = detail_check(10, load_data_path, False, review_data_path, False, condition)

In [0]:

load_data_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_201908_20200116_1/"
review_data_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/"
df_review_data = spark.read.parquet(review_data_path).where("event_month='2019-10'")
df_review_data.where("app_id='1084930849' and country_code='CA' and review_id='4866884041'").show()

In [0]:

print [i['review_id'] for i in row_picked_data]
print [i['content'] for i in row_source_data]==[i['content'] for i in row_picked_data]
print "Done"

In [0]:

def detail_check(num, picked_data_path, delta1, condition1, source_data_path, delta2, condition2):
    def read_data(path, delta, condition):
        '''
        check data format (delta lake or not)
        '''
        if delta:
            df = spark.read.format('delta').load(path)
        df = spark.read.parquet(path)
        if condition:
            return df.where(condition)
        return df
        
    # Read data from s3 path
    df_picked_data = read_data(picked_data_path, delta1, condition1)
    df_source_data = read_data(source_data_path, delta2, condition2)
    
    # Review id list from picked data
    row_picked_data = df_picked_data.take(10)
    
    print "row_picked_data:", row_picked_data
    
    review_id_list = [row['review_id'] for row in row_picked_data]
    
    print "review_id_list:", review_id_list
    
    # Search picked data in source data
    row_source_data = []
    for review_id in review_id_list:
        print "review_id = ", review_id
        row_source_data.append(df_source_data.where("review_id='{}'".format(review_id)).collect()[0])
        print row_source_data
    
    return df_picked_data, df_source_data, row_picked_data, row_source_data
    

load_data_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200116_2020024_2/"
review_data_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/"
condition1 = "event_month='2020-01'"
condition2 = "process_date>='2020-01-01' and process_date<='2020-02-01'"
df_picked_data_b, df_source_data_b, row_picked_data_b, row_source_data_b = detail_check(10, load_data_path, False, condition1, review_data_path, False, condition2)


In [0]:

print [i['review_id'] for i in row_picked_data_b]
print [i['content'] for i in row_source_data_b]==[i['content'] for i in row_picked_data_b]
print "Done"

In [0]:

def detail_check(num, picked_data_path, delta1, condition1, source_data_path, delta2, condition2):
    def read_data(path, delta, condition):
        '''
        check data format (delta lake or not)
        '''
        if delta:
            df = spark.read.format('delta').load(path)
        df = spark.read.parquet(path)
        if condition:
            return df.where(condition)
        return df
        
    # Read data from s3 path
    df_picked_data = read_data(picked_data_path, delta1, condition1)
    df_source_data = read_data(source_data_path, delta2, condition2)
    
    # Review id list from picked data
    row_picked_data = df_picked_data.take(10)
    
    print "row_picked_data:", row_picked_data
    
    review_id_list = [row['review_id'] for row in row_picked_data]
    
    print "review_id_list:", review_id_list
    
    # Search picked data in source data
    row_source_data = []
    for review_id in review_id_list:
        print "review_id = ", review_id
        row_source_data.append(df_source_data.where("review_id='{}'".format(review_id)).collect())
        print row_source_data[-1]
    
    return df_picked_data, df_source_data, row_picked_data, row_source_data
    

load_data_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200220_202003_3/"
review_data_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/"
condition1 = "event_month='2020-02'"
condition2 = "process_date>='2020-02-01' and process_date<='2020-03-01'"
df_picked_data_c, df_source_data_c, row_picked_data_c, row_source_data_c = detail_check(10, load_data_path, False, condition1, review_data_path, False, condition2)


In [0]:

a = spark.read.parquet(review_data_path).where("review_id='5501297572'")
a.show()

In [0]:

a.where("time='2020-02*'").show()

In [0]:

print [i['review_id'] for i in row_picked_data_c]
print [i['content'] for i in row_source_data_c]==[i['content'] for i in row_picked_data_c]
print "Done"

In [0]:

load_data_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/_obsolete/aa.adv_join_20200116_2020024_2/"
review_data_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/"
condition = "process_date>='2020-02-20' and process_date<='2020-03-07'"
df_picked_data_c, df_source_data_c, row_picked_data_c, row_source_data_c = detail_check(10, load_data_path, False, review_data_path, True)
